In [17]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import sklearn.metrics

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

In [18]:
df = pd.read_csv('./project_data/out_eda1.csv', 
                 index_col = 'Claim Identifier')

In [107]:
test = pd.read_csv('./project_data/test_treated', index_col = 'Claim Identifier')

In [108]:
#df[var_name].fillna(df[var_name].median())

**Missing Values**

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import RobustScaler


In [26]:
def ball_tree_impute(df, target, n_neighbors=5):
    # Get all features except the target column
    features = df.columns.drop([target, 'Claim Injury Type'])
    
    # Separate rows with and without missing target values
    missing_mask = df[target].isna()
    non_missing_data = df[~missing_mask]
    missing_data = df[missing_mask]

    # Build a ball tree using all features except the target column
    knn = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ball_tree')
    knn.fit(non_missing_data[features])

    # Find nearest neighbors for rows with missing values
    _, indices = knn.kneighbors(missing_data[features])

    # Impute missing values by averaging the target values of nearest neighbors
    imputed_values = [
        non_missing_data.iloc[neighbor_indices][target].mean() for neighbor_indices in indices
    ]
    
    # Assign the imputed values to the missing target values in the original DataFrame
    df.loc[missing_mask, target] = imputed_values

    return df

In [22]:
for col in df.columns:
    df = custom_impute(df, col)

In [27]:
df = ball_tree_impute(df, 'Average Weekly Wage')
df
# 49 seconds, yay

,Age at Injury,Alternative Dispute Resolution,Attorney/Representative,Average Weekly Wage,Birth Year,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,Gender,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Zip Code,Number of Dependents,Accident Year,Accident Month,Accident Day,Assembly Year,Assembly Month,Assembly Day,C-2 Year,C-2 Month,C-2 Day,C-3 Date Binary,First Hearing Year
Claim Identifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5393875,31.0,571411.0,0.0,0.000,1988.0,12736.0,285367.0,2. NON-COMP,3355.0,0.0,44646.0,0.0,0.0,44.0,135885.0,27.0,10.0,62.0,13662.0,1.0,2019.0,12.0,30.0,2020,1,1,2019.0,12.0,31.0,0,0
5393091,46.0,571411.0,1.0,1745.930,1973.0,2862.0,285367.0,4. TEMPORARY,760.0,0.0,40449.0,1.0,4.0,23.0,135885.0,97.0,49.0,38.0,14569.0,4.0,2019.0,8.0,30.0,2020,1,1,2020.0,1.0,1.0,1,2020
5393889,40.0,571411.0,0.0,1434.800,1979.0,9126.0,285367.0,4. TEMPORARY,17450.0,0.0,86171.0,0.0,0.0,56.0,85033.0,79.0,7.0,10.0,12589.0,6.0,2019.0,12.0,6.0,2020,1,1,2020.0,1.0,1.0,0,0
5393887,61.0,571411.0,0.0,325.484,1958.0,111144.0,111144.0,2. NON-COMP,11530.0,0.0,86171.0,0.0,0.0,62.0,85033.0,16.0,43.0,36.0,12603.0,1.0,2019.0,12.0,30.0,2020,1,1,2019.0,12.0,31.0,0,0
5393863,67.0,571411.0,0.0,0.000,1952.0,14366.0,285367.0,3. MED ONLY,60430.0,0.0,60536.0,0.0,0.0,44.0,265981.0,31.0,10.0,38.0,11772.0,5.0,2019.0,12.0,26.0,2020,1,1,2019.0,12.0,31.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6165265,35.0,571411.0,0.0,0.000,1987.0,111144.0,111144.0,2. NON-COMP,39618.0,0.0,270778.0,0.0,0.0,56.0,265981.0,45.0,10.0,42.0,10467.0,4.0,2022.0,12.0,14.0,2022,12,30,2022.0,12.0,30.0,0,0
6165285,33.0,571411.0,0.0,0.000,1989.0,2331.0,285367.0,2. NON-COMP,43580.0,0.0,270778.0,0.0,0.0,62.0,265981.0,74.0,37.0,35.0,11590.0,6.0,2022.0,12.0,15.0,2022,12,31,2022.0,12.0,31.0,0,0
6165506,61.0,571411.0,0.0,991.080,1961.0,2950.0,285367.0,4. TEMPORARY,30874.0,0.0,45605.0,1.0,1.0,62.0,85033.0,98.0,80.0,34.0,14227.0,3.0,2022.0,12.0,13.0,2022,12,31,2022.0,12.0,31.0,0,0


**Outliers**

In [111]:
df = df.dropna()

In [112]:
df.isna().sum()

Age at Injury                     0
Alternative Dispute Resolution    0
Attorney/Representative           0
Average Weekly Wage               0
Birth Year                        0
Carrier Name                      0
Carrier Type                      0
Claim Injury Type                 0
County of Injury                  0
COVID-19 Indicator                0
District Name                     0
Gender                            0
IME-4 Count                       0
Industry Code                     0
Medical Fee Region                0
WCIO Cause of Injury Code         0
WCIO Nature of Injury Code        0
WCIO Part Of Body Code            0
Zip Code                          0
Number of Dependents              0
Accident Year                     0
Accident Month                    0
Accident Day                      0
Assembly Year                     0
Assembly Month                    0
Assembly Day                      0
C-2 Year                          0
C-2 Month                   

In [24]:
begin = df.index[df['Birth Year'].isna()].tolist()
df = custom_impute(df, 'Birth Year')
after = df.index[df['Birth Year'].isna()].tolist()

In [33]:
[i for i, value in enumerate(after) if value not in begin]

[]

In [110]:
for col in df.columns:
    df = custom_impute(df, col)

In [14]:
df[df['Accident Year'].notna() & df['Age at Injury'].notna() & df['Birth Year'].isna()]

,Age at Injury,Alternative Dispute Resolution,Attorney/Representative,Average Weekly Wage,Birth Year,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,Gender,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Zip Code,Number of Dependents,Accident Year,Accident Month,Accident Day,Assembly Year,Assembly Month,Assembly Day,C-2 Year,C-2 Month,C-2 Day,C-3 Date Binary,First Hearing Year
Claim Identifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5395789,0.0,571411.0,1.0,0.0,NaN,942.0,52788.0,1,39618.0,0.0,270778.0,0.0,0.0,45.0,265981.0,98.0,52.0,22.0,10456,6.0,2021.0,7.0,15.0,2020,1,6,2020.0,3.0,9.0,1,2020
5400472,0.0,571411.0,1.0,0.0,NaN,11038.0,285367.0,1,58333.0,0.0,270778.0,0.0,0.0,51.0,265981.0,52.0,72.0,13.0,11354,2.0,2021.0,7.0,15.0,2020,1,10,2020.0,2.0,19.0,1,2020
5401700,0.0,571411.0,1.0,0.0,NaN,17540.0,285367.0,1,53207.0,0.0,270778.0,1.0,2.0,56.0,265981.0,99.0,59.0,42.0,11214,1.0,2021.0,7.0,15.0,2020,1,13,2020.0,2.0,10.0,1,2020
5401542,0.0,571411.0,1.0,0.0,NaN,469.0,285367.0,1,60430.0,0.0,60536.0,0.0,0.0,23.0,265981.0,98.0,31.0,13.0,11730,2.0,2021.0,7.0,15.0,2020,1,13,2021.0,4.0,13.0,1,2021
5408856,0.0,571411.0,1.0,0.0,NaN,8224.0,121920.0,1,53207.0,0.0,270778.0,1.0,0.0,48.0,265981.0,99.0,31.0,13.0,11236,0.0,2021.0,7.0,15.0,2020,1,22,2020.0,1.0,30.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6135328,0.0,571411.0,1.0,0.0,NaN,136.0,121920.0,0,30874.0,0.0,45605.0,0.0,0.0,0.0,85033.0,0.0,0.0,0.0,14221,5.0,2021.0,7.0,15.0,2022,11,18,2021.0,7.0,16.0,1,0
6138100,0.0,571411.0,1.0,NaN,NaN,8224.0,121920.0,1,30014.0,0.0,270778.0,0.0,0.0,48.0,265981.0,99.0,31.0,13.0,10003,6.0,2021.0,7.0,15.0,2022,11,21,2022.0,11.0,30.0,1,0
6150079,0.0,571411.0,1.0,0.0,NaN,1258.0,285367.0,0,7248.0,0.0,45605.0,0.0,0.0,44.0,135885.0,0.0,0.0,0.0,14092,1.0,2021.0,7.0,15.0,2022,12,9,2021.0,7.0,16.0,1,0


In [66]:
len(df)

574025

In [43]:
df[df['Birth Year'].isna()]

,Age at Injury,Alternative Dispute Resolution,Attorney/Representative,Average Weekly Wage,Birth Year,Carrier Name,Carrier Type,Claim Injury Type,County of Injury,COVID-19 Indicator,District Name,Gender,IME-4 Count,Industry Code,Medical Fee Region,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Zip Code,Number of Dependents,Accident Year,Accident Month,Accident Day,Assembly Year,Assembly Month,Assembly Day,C-2 Year,C-2 Month,C-2 Day,C-3 Date Binary,First Hearing Year
Claim Identifier,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5395789,0.0,571411.0,1.0,0.0,NaN,942.0,52788.0,1,39618.0,0.0,270778.0,0.0,0.0,45.0,265981.0,98.0,52.0,22.0,10456,6.0,2021.0,7.0,15.0,2020,1,6,2020.0,3.0,9.0,1,2020
5400472,0.0,571411.0,1.0,0.0,NaN,11038.0,285367.0,1,58333.0,0.0,270778.0,0.0,0.0,51.0,265981.0,52.0,72.0,13.0,11354,2.0,2021.0,7.0,15.0,2020,1,10,2020.0,2.0,19.0,1,2020
5401700,0.0,571411.0,1.0,0.0,NaN,17540.0,285367.0,1,53207.0,0.0,270778.0,1.0,2.0,56.0,265981.0,99.0,59.0,42.0,11214,1.0,2021.0,7.0,15.0,2020,1,13,2020.0,2.0,10.0,1,2020
5401542,0.0,571411.0,1.0,0.0,NaN,469.0,285367.0,1,60430.0,0.0,60536.0,0.0,0.0,23.0,265981.0,98.0,31.0,13.0,11730,2.0,2021.0,7.0,15.0,2020,1,13,2021.0,4.0,13.0,1,2021
5408856,0.0,571411.0,1.0,0.0,NaN,8224.0,121920.0,1,53207.0,0.0,270778.0,1.0,0.0,48.0,265981.0,99.0,31.0,13.0,11236,0.0,2021.0,7.0,15.0,2020,1,22,2020.0,1.0,30.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6135328,0.0,571411.0,1.0,0.0,NaN,136.0,121920.0,0,30874.0,0.0,45605.0,0.0,0.0,0.0,85033.0,0.0,0.0,0.0,14221,5.0,2021.0,7.0,15.0,2022,11,18,2021.0,7.0,16.0,1,0
6138100,0.0,571411.0,1.0,NaN,NaN,8224.0,121920.0,1,30014.0,0.0,270778.0,0.0,0.0,48.0,265981.0,99.0,31.0,13.0,10003,6.0,2021.0,7.0,15.0,2022,11,21,2022.0,11.0,30.0,1,0
6150079,0.0,571411.0,1.0,0.0,NaN,1258.0,285367.0,0,7248.0,0.0,45605.0,0.0,0.0,44.0,135885.0,0.0,0.0,0.0,14092,1.0,2021.0,7.0,15.0,2022,12,9,2021.0,7.0,16.0,1,0


## 1. K Fold

<a href="#top">Top &#129033;</a>

In [113]:
X = df.drop('Claim Injury Type', axis = 1)
y = df['Claim Injury Type']

In [114]:
X.isna().sum()

Age at Injury                     0
Alternative Dispute Resolution    0
Attorney/Representative           0
Average Weekly Wage               0
Birth Year                        0
Carrier Name                      0
Carrier Type                      0
County of Injury                  0
COVID-19 Indicator                0
District Name                     0
Gender                            0
IME-4 Count                       0
Industry Code                     0
Medical Fee Region                0
WCIO Cause of Injury Code         0
WCIO Nature of Injury Code        0
WCIO Part Of Body Code            0
Zip Code                          0
Number of Dependents              0
Accident Year                     0
Accident Month                    0
Accident Day                      0
Assembly Year                     0
Assembly Month                    0
Assembly Day                      0
C-2 Year                          0
C-2 Month                         0
C-2 Day                     

In [115]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)

In [116]:
predictions = []

for train_index, val_index in kf.split(X):
    
    # Split data into training and validation folds
    X_train = X.iloc[train_index]
    X_val = X.iloc[val_index]
    y_train = y.iloc[train_index]
    y_val = y.iloc[val_index]
    
    # missing values
    for col in X_train.columns:
        X_train = custom_impute(X_train, col)
        
    for col in X_val.columns:
        X_val = custom_impute(X_val, col)
        
        
    # outliers (log) & scaler (robust)
    
#     preprocessor = Pipeline(steps=[
#         ('log_transform', FunctionTransformer(func=log_transform, validate=False)),
#         ('scaler', RobustScaler())
#     ])
    
#     X_train = preprocessor.fit_transform(X_train)
#     X_val = preprocessor.transform(X_val)
    
    
    # feature selection - selecionar só colunas boas antes
    
    
    
    # fit model
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    # make predictions
    train_pred = model.predict(X_train)
    val_pred = model.predict(X_val)

    # compute metrics
    print(classification_report(y_train, train_pred))
    print(classification_report(y_val, val_pred))
    
    # save predictions and best model's parameters
    
    predictions.append({'Train Predictions': train_pred, 'Validation Predictions': val_pred})


                 precision    recall  f1-score   support

   1. CANCELLED       0.00      0.00      0.00      9906
    2. NON-COMP       0.60      0.98      0.74    232910
    3. MED ONLY       0.00      0.00      0.00     55214
   4. TEMPORARY       0.56      0.32      0.41    118608
5. PPD SCH LOSS       0.16      0.05      0.07     38661
     6. PPD NSL       0.00      0.00      0.00      3336
         7. PTD       0.00      0.00      0.00        69
       8. DEATH       0.02      0.00      0.00       375

       accuracy                           0.58    459079
      macro avg       0.17      0.17      0.15    459079
   weighted avg       0.46      0.58      0.49    459079

                 precision    recall  f1-score   support

   1. CANCELLED       0.00      0.00      0.00      2512
    2. NON-COMP       0.60      0.98      0.74     58063
    3. MED ONLY       0.00      0.00      0.00     13687
   4. TEMPORARY       0.56      0.32      0.41     29891
5. PPD SCH LOSS       0.15 

## test

In [117]:
for col in test.columns:
    test = custom_impute(test, col)

In [118]:
test = test.fillna(0)
test.isna().sum()

Age at Injury                     0
Alternative Dispute Resolution    0
Attorney/Representative           0
Average Weekly Wage               0
Birth Year                        0
Carrier Name                      0
Carrier Type                      0
County of Injury                  0
COVID-19 Indicator                0
District Name                     0
Gender                            0
IME-4 Count                       0
Industry Code                     0
Medical Fee Region                0
WCIO Cause of Injury Code         0
WCIO Nature of Injury Code        0
WCIO Part Of Body Code            0
Zip Code                          0
Number of Dependents              0
Accident Year                     0
Accident Month                    0
Accident Day                      0
Assembly Year                     0
Assembly Month                    0
Assembly Day                      0
C-2 Year                          0
C-2 Month                         0
C-2 Day                     

In [122]:
test['Claim Injury Type'] = model.predict(test)


In [124]:
test['Claim Injury Type'].value_counts() 

Claim Injury Type
2. NON-COMP        345305
4. TEMPORARY        33482
5. PPD SCH LOSS      9132
1. CANCELLED           52
6. PPD NSL              4
Name: count, dtype: int64

In [125]:
predictions = test['Claim Injury Type']
predictions

Claim Identifier
6165911    2. NON-COMP
6166141    2. NON-COMP
6165907    2. NON-COMP
6166047    2. NON-COMP
6166102    2. NON-COMP
              ...     
6553137    2. NON-COMP
6553119    2. NON-COMP
6553542    2. NON-COMP
6553455    2. NON-COMP
6553594    2. NON-COMP
Name: Claim Injury Type, Length: 387975, dtype: object

In [128]:
predictions.to_csv('./predictions/pred1.csv')